In [ ]:
#@title Install Dependencies
%%capture
!pip install -q rdkit gradio==4.44.1

In [ ]:
#@title Install gnina
import os
import stat
import requests

vina_dir = os.path.join(os.path.basename(''), 'working_dir', 'vina')
os.makedirs(vina_dir)
gnina_pth = os.path.join(vina_dir, 'gnina')
with requests.get("https://github.com/gnina/gnina/releases/download/v1.3/gnina", stream=True) as response:
    with open(gnina_pth, 'wb') as file:
        for chunk in response.iter_content(chunk_size=8192):
            file.write(chunk)
st = os.stat(gnina_pth)
os.chmod(gnina_pth, st.st_mode | stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH)
print('gnina downloaded!')

In [ ]:
#@title Gradio Interface
import os
import re
import json
import pickle
import shutil
import zipfile
import tempfile
import subprocess
import multiprocessing
import numpy as np
import gradio as gr
import pandas as pd

from rdkit import Chem
from os import linesep as os_linesep

from concurrent.futures import ProcessPoolExecutor, as_completed

#@title Params
residue_params_json = """{
  "ALA": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB1",   "HB2",   "HB3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.168,   0.000,  -0.344,   0.164,   0.041,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "CYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG",    "HG"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.181,   0.000,  -0.342,   0.164,   0.101,   0.000,   0.000,  -0.177,   0.102],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10]],
    "bond_cut_atoms": [0, 4]
  },
  "CYX": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.244,  -0.272,   0.183,   0.000,  -0.343,   0.164,   0.116,   0.000,   0.000,  -0.092],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [0, 4, 9]
  },
  "CYM": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.240,  -0.273,   0.147,   0.000,  -0.346,   0.164,  -0.144,   0.000,   0.000,  -0.788],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "ASP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.176,   0.000,  -0.343,   0.164,   0.087,   0.000,   0.000,   0.044,  -0.548,  -0.549],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11]],
    "bond_cut_atoms": [0, 4]
  },
  "ASH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,     1.9,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,     5.0,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.182,   0.000,  -0.342,   0.164,   0.157,   0.000,   0.000,   0.305,  -0.252,  -0.479,   0.296],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12]],
    "bond_cut_atoms": [0, 4]
  },
  "GLU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.171,   0.000,  -0.344,   0.164,   0.041,   0.000,   0.000,   0.055,   0.000,   0.000,   0.041,  -0.548,  -0.549],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14]],
    "bond_cut_atoms": [0, 4]
  },
  "GLH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2",   "HE2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.171,   0.000,  -0.344,   0.164,   0.047,   0.000,   0.000,   0.125,   0.000,   0.000,   0.303,  -0.252,  -0.479,   0.296],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [14, 15]],
    "bond_cut_atoms": [0, 4]
  },
  "PHE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "HZ",   "CE2",   "HE2",   "CD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false,    true,   false],
         "gasteiger": [   0.243,  -0.272,   0.175,   0.000,  -0.343,   0.164,   0.070,   0.000,   0.000,  -0.045,   0.004,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.004,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [14, 16], [16, 17], [16, 18], [18, 19], [18, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "GLY": {
        "atom_names": [     "C",     "O",    "CA",   "HA2",     "N",     "H",   "HA3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.239,  -0.273,   0.216,   0.000,  -0.346,   0.164,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6]],
    "bond_cut_atoms": [0, 4]
  },
  "HIE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",    "NA",     "A",     "H",     "N",    "HD",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,    null,    null,    null,    true,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,    null,    null,    null,    true,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,     1.9,    null,     1.8,    null,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.272,   0.177,   0.000,  -0.343,   0.164,   0.093,   0.000,   0.000,   0.061,  -0.242,   0.196,   0.000,  -0.350,   0.167,   0.107,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [11, 12], [11, 13], [13, 14], [13, 15], [15, 16], [15, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "HID": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "CD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",    "NA",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,     1.9,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,     5.0,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.272,   0.177,   0.000,  -0.343,   0.164,   0.090,   0.000,   0.000,   0.037,  -0.347,   0.167,   0.196,   0.000,  -0.245,   0.134,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [15, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "HIP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",     "N",    "HD",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,    null,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.179,   0.000,  -0.343,   0.164,   0.119,   0.000,   0.000,   0.142,  -0.248,   0.312,   0.400,   0.000,  -0.250,   0.312,   0.244,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [14, 16], [16, 17], [16, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "ILE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "CG1",  "HG12",  "HG13",   "CD1",  "HD11",  "HD12",  "HD13"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.174,   0.000,  -0.343,   0.164,   0.018,   0.000,   0.010,   0.000,   0.000,   0.000,   0.002,   0.000,   0.000,   0.004,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18]],
    "bond_cut_atoms": [0, 4]
  },
  "LYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ1",   "HZ2",   "HZ3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.171,   0.000,  -0.344,   0.164,   0.035,   0.000,   0.000,   0.008,   0.000,   0.000,   0.061,   0.000,   0.000,   0.258,   0.000,   0.000,  -0.357,   0.345,   0.345,   0.345],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18], [18, 19], [18, 20], [18, 21]],
    "bond_cut_atoms": [0, 4]
  },
  "LYN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ2",   "HZ3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",    "NA",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.171,   0.000,  -0.344,   0.164,   0.035,   0.000,   0.000,   0.004,   0.000,   0.000,   0.015,   0.000,   0.000,   0.077,   0.000,   0.000,  -0.329,   0.118,   0.118],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18], [18, 19], [18, 20]],
    "bond_cut_atoms": [0, 4]
  },
  "LEU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",    "HG",   "CD1",  "HD11",  "HD12",  "HD13",   "CD2",  "HD21",  "HD22",  "HD23"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.172,   0.000,  -0.345,   0.165,   0.038,   0.000,   0.000,  -0.015,   0.000,   0.007,   0.000,   0.000,   0.000,   0.007,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12], [11, 13], [11, 14], [9, 15], [15, 16], [15, 17], [15, 18]],
    "bond_cut_atoms": [0, 4]
  },
  "MET": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "SD",    "CE",   "HE1",   "HE2",   "HE3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",    "SA",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.20,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00214,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     2.5,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     1.0,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     2.0,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.272,   0.172,   0.000,  -0.344,   0.164,   0.046,   0.000,   0.000,   0.072,   0.000,   0.000,  -0.165,   0.085,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [13, 14], [13, 15], [13, 16]],
    "bond_cut_atoms": [0, 4]
  },
  "ASN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "ND2",  "HD22",  "HD21"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,    null,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,    null,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.180,   0.000,  -0.342,   0.164,   0.134,   0.000,   0.000,   0.220,  -0.275,  -0.368,   0.159,   0.159],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12], [11, 13]],
    "bond_cut_atoms": [0, 4]
  },
  "PRO": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null],
        "vina_donor": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false],
         "gasteiger": [   0.243,  -0.271,   0.175,   0.000,  -0.329,   0.037,   0.000,   0.000,   0.024,   0.000,   0.000,   0.121,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [11, 4]],
    "bond_cut_atoms": [0, 4]
  },
  "GLN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "NE2",  "HE22",  "HE21"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,    null,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,    null,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.171,   0.000,  -0.343,   0.164,   0.045,   0.000,   0.000,   0.102,   0.000,   0.000,   0.217,  -0.275,  -0.368,   0.159,   0.159],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [14, 15], [14, 16]],
    "bond_cut_atoms": [0, 4]
  },
  "ARG": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "NE",    "HE",    "CZ",   "NH1",  "HH12",  "HH11",   "NH2",  "HH22",  "HH21"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",     "C",     "N",    "HD",    "HD",     "N",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.15,    0.16,    0.02,    0.02,    0.16,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    4.00,    3.50,    2.00,    2.00,    3.50,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000, 33.5103, 22.4493,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051,-0.00143,-0.00162, 0.00051, 0.00051,-0.00162, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,     1.9,     1.8,    null,    null,     1.8,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.242,  -0.273,   0.171,   0.000,  -0.344,   0.164,   0.038,   0.000,   0.000,   0.040,   0.000,   0.000,   0.197,   0.000,   0.000,  -0.278,   0.260,   0.339,  -0.290,   0.256,   0.256,  -0.290,   0.256,   0.256],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [17, 18], [18, 19], [18, 20], [17, 21], [21, 22], [21, 23]],
    "bond_cut_atoms": [0, 4]
  },
  "SER": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "OG",    "HG"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "OA",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    3.20,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 17.1573,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00251, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     1.9,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     5.0,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.7,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.244,  -0.272,   0.197,   0.000,  -0.342,   0.164,   0.191,   0.000,   0.000,  -0.392,   0.210],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10]],
    "bond_cut_atoms": [0, 4]
  },
  "THR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "OG1",   "HG1"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",    "OA",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     1.9,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     5.0,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.7,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
         "gasteiger": [   0.245,  -0.273,   0.199,   0.000,  -0.341,   0.164,   0.143,   0.000,   0.041,   0.000,   0.000,   0.000,  -0.389,   0.211],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13]],
    "bond_cut_atoms": [0, 4]
  },
  "VAL": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG1",  "HG11",  "HG12",  "HG13",   "CG2",  "HG21",  "HG22",  "HG23"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false],
         "gasteiger": [   0.243,  -0.272,   0.173,   0.000,  -0.343,   0.164,   0.015,   0.000,   0.010,   0.000,   0.000,   0.000,   0.010,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13], [12, 14], [12, 15]],
    "bond_cut_atoms": [0, 4]
  },
  "TRP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "NE1",   "HE1",   "CE2",   "CD2",   "CE3",   "HE3",   "CZ3",   "HZ3",   "CH2",   "HH2",   "CZ2",   "HZ2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "N",    "HD",     "A",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.16,    0.02,    0.15,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    3.50,    2.00,    4.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.8,    null,     1.9,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false],
         "gasteiger": [   0.242,  -0.272,   0.175,   0.000,  -0.343,   0.164,   0.072,   0.000,   0.000,  -0.020,   0.087,   0.000,  -0.359,   0.166,   0.046,   0.003,   0.010,   0.000,   0.001,   0.000,   0.002,   0.000,   0.026,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [16, 17], [16, 18], [18, 19], [18, 20], [20, 21], [20, 22], [22, 23], [15, 9], [22, 14]],
    "bond_cut_atoms": [0, 4]
  },
  "TYR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "OH",    "HH",   "CE2",   "HE2",   "CD2",   "HD2"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",    "OA",    "HD",     "A",     "H",     "A",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.20,    0.02,    0.15,    0.00,    0.15,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    3.20,    2.00,    4.00,    0.00,    4.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103, 17.1573,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052,-0.00251, 0.00051,-0.00052, 0.00000,-0.00052, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,     1.7,    null,     1.9,    null,     1.9,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,   false,   false,   false,    true,   false,    true,   false],
         "gasteiger": [   0.243,  -0.272,   0.175,   0.000,  -0.343,   0.164,   0.070,   0.000,   0.000,  -0.045,   0.007,   0.000,   0.046,   0.000,   0.115,  -0.507,   0.294,   0.046,   0.000,   0.007,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [14, 17], [17, 18], [17, 19], [19, 20], [19, 9]],
    "bond_cut_atoms": [0, 4]
  },
  "NME": {
        "atom_names": [   "CH3",  "HH31",  "HH32",  "HH33",     "N",     "H"],
        "atom_types": [     "C",     "H",     "H",     "H",     "N",    "HD"],
         "ad4_epsii": [    0.15,    0.00,    0.00,    0.00,    0.16,    0.02],
           "ad4_rii": [    4.00,    0.00,    0.00,    0.00,    3.50,    2.00],
       "ad4_sol_vol": [ 33.5103,  0.0000,  0.0000,  0.0000, 22.4493,  0.0000],
       "ad4_sol_par": [-0.00143, 0.00000, 0.00000, 0.00000,-0.00162, 0.00051],
        "ad4_hb_rij": [    null,    null,    null,    null,    null,    true],
      "ad4_hb_epsij": [    null,    null,    null,    null,    null,    true],
           "vina_ri": [     1.9,    null,    null,    null,     1.8,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false],
     "vina_acceptor": [   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.154,   0.000,   0.000,   0.000,  -0.330,   0.176],
             "bonds": [[0, 1], [0, 2], [0, 3], [0, 4], [4, 5]],
    "bond_cut_atoms": [4]
  },
  "ACE": {
        "atom_names": [     "C",     "O",   "CH3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,    true,   false,   false,   false],
         "gasteiger": [   0.196,  -0.300,   0.104,   0.000,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [2, 5]],
    "bond_cut_atoms": [0]
  },
  "NPRO": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "H2",    "H3",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false],
         "gasteiger": [   0.271,  -0.274,   0.266,   0.000,  -0.343,   0.339,   0.339,   0.079,   0.000,   0.000,   0.067,   0.000,   0.000,   0.256,   0.000,   0.000],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [4, 6], [2, 7], [7, 8], [7, 9], [7, 10], [10, 11], [10, 12], [10, 13], [13, 14], [13, 15], [13, 4]],
    "bond_cut_atoms": [0]
  },
  "NALA": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB1",   "HB2",   "HB3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.271,  -0.274,   0.261,   0.000,  -0.356,   0.084,   0.000,   0.000,   0.000,   0.338,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 9], [4, 10], [4, 11], [2, 5], [5, 6], [5, 7], [5, 8]],
    "bond_cut_atoms": [0]
  },
  "NCYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "SG",    "HG",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",    "SA",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.20,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00214, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,     2.5,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,     1.0,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     2.0,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.273,   0.275,   0.000,  -0.354,   0.138,   0.000,   0.000,  -0.176,   0.100,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 10], [4, 11], [4, 12], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9]],
    "bond_cut_atoms": [0]
  },
  "NCYX": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "SG",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",    "SA",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.20,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00214, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,     2.5,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,     1.0,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     2.0,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.273,   0.275,   0.000,  -0.353,   0.154,   0.000,   0.000,  -0.090,   0.338,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 9], [4, 10], [4, 11], [2, 5], [5, 6], [5, 7], [5, 8]],
    "bond_cut_atoms": [0, 8]
  },
  "NCYM": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "SG",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",    "SA",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.20,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00214, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,     2.5,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,     1.0,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     2.0,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.271,  -0.272,   0.242,   0.000,  -0.355,  -0.105,   0.000,   0.000,  -0.798,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 9], [4, 10], [4, 11], [2, 5], [5, 6], [5, 7], [5, 8]],
    "bond_cut_atoms": [0]
  },
  "NASP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.272,   0.000,  -0.352,   0.126,   0.000,   0.000,   0.047,  -0.557,  -0.558,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 11], [4, 12], [4, 13], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10]],
    "bond_cut_atoms": [0]
  },
  "NASH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.278,   0.000,  -0.351,   0.194,   0.000,   0.000,   0.305,  -0.255,  -0.486,   0.292,   0.341,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 12], [4, 13], [4, 14], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [10, 11]],
    "bond_cut_atoms": [0]
  },
  "NGLU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.272,   0.267,   0.000,  -0.352,   0.080,   0.000,   0.000,   0.058,   0.000,   0.000,   0.041,  -0.557,  -0.558,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 14], [4, 15], [4, 16], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13]],
    "bond_cut_atoms": [0]
  },
  "NGLH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2",   "HE2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.272,   0.267,   0.000,  -0.351,   0.086,   0.000,   0.000,   0.127,   0.000,   0.000,   0.299,  -0.255,  -0.486,   0.292,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 15], [4, 16], [4, 17], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [13, 14]],
    "bond_cut_atoms": [0]
  },
  "NPHE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "HZ",   "CE2",   "HE2",   "CD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.272,  -0.274,   0.269,   0.000,  -0.355,   0.109,   0.000,   0.000,  -0.043,   0.004,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.004,   0.000,   0.338,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 19], [4, 20], [4, 21], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [9, 11], [11, 12], [11, 13], [13, 14], [13, 15], [15, 16], [15, 17], [17, 18], [17, 8]],
    "bond_cut_atoms": [0]
  },
  "NGLY": {
        "atom_names": [     "C",     "O",    "CA",   "HA2",     "N",   "HA3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.268,  -0.275,   0.353,   0.000,  -0.357,   0.000,   0.337,   0.337,   0.337],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 6], [4, 7], [4, 8], [2, 5]],
    "bond_cut_atoms": [0]
  },
  "NHIE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",    "NA",     "A",     "H",     "N",    "HD",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.16,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.50,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,     1.9,    null,    null,    null,    true,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,     5.0,    null,    null,    null,    true,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.8,     1.9,    null,     1.8,    null,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.272,   0.000,  -0.352,   0.131,   0.000,   0.000,   0.064,  -0.245,   0.193,   0.000,  -0.355,   0.165,   0.105,   0.000,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 16], [4, 17], [4, 18], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [14, 8]],
    "bond_cut_atoms": [0]
  },
  "NHID": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "CD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",    "NA",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,     1.9,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,     5.0,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.272,   0.000,  -0.352,   0.129,   0.000,   0.000,   0.040,  -0.352,   0.165,   0.193,   0.000,  -0.249,   0.132,   0.000,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 16], [4, 17], [4, 18], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [9, 11], [11, 12], [11, 13], [13, 14], [14, 15], [14, 8]],
    "bond_cut_atoms": [0]
  },
  "NHIP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",     "N",    "HD",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,    null,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.275,  -0.271,   0.276,   0.000,  -0.351,   0.157,   0.000,   0.000,   0.144,  -0.250,   0.307,   0.395,   0.000,  -0.253,   0.307,   0.241,   0.000,   0.341,   0.341,   0.341],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 17], [4, 18], [4, 19], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [9, 11], [11, 12], [11, 13], [13, 14], [13, 15], [15, 16], [15, 8]],
    "bond_cut_atoms": [0]
  },
  "NILE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "CG1",  "HG12",  "HG13",   "CD1",  "HD11",  "HD12",  "HD13",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,     1.9,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.272,  -0.274,   0.267,   0.000,  -0.355,   0.053,   0.000,   0.014,   0.000,   0.000,   0.000,   0.006,   0.000,   0.000,   0.004,   0.000,   0.000,   0.000,   0.337,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 18], [4, 19], [4, 20], [2, 5], [5, 6], [5, 7], [7, 8], [7, 9], [7, 10], [5, 11], [11, 12], [11, 13], [11, 14], [14, 15], [14, 16], [14, 17]],
    "bond_cut_atoms": [0]
  },
  "NLYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ1",   "HZ2",   "HZ3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",    "HD",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.271,   0.267,   0.000,  -0.352,   0.075,   0.000,   0.000,   0.012,   0.000,   0.000,   0.060,   0.000,   0.000,   0.255,   0.000,   0.000,  -0.363,   0.340,   0.340,   0.340,   0.341,   0.341,   0.341],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 21], [4, 22], [4, 23], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [11, 14], [14, 15], [14, 16], [14, 17], [17, 18], [17, 19], [17, 20]],
    "bond_cut_atoms": [0]
  },
  "NLYN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ2",   "HZ3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",    "NA",    "HD",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.272,  -0.272,   0.266,   0.000,  -0.353,   0.074,   0.000,   0.000,   0.008,   0.000,   0.000,   0.015,   0.000,   0.000,   0.076,   0.000,   0.000,  -0.335,   0.116,   0.116,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 20], [4, 21], [4, 22], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [11, 14], [14, 15], [14, 16], [14, 17], [17, 18], [17, 19]],
    "bond_cut_atoms": [0]
  },
  "NLEU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",    "HG",   "CD1",  "HD11",  "HD12",  "HD13",   "CD2",  "HD21",  "HD22",  "HD23",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.271,  -0.274,   0.265,   0.000,  -0.355,   0.076,   0.000,   0.000,  -0.011,   0.000,   0.007,   0.000,   0.000,   0.000,   0.007,   0.000,   0.000,   0.000,   0.338,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 18], [4, 19], [4, 20], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [10, 11], [10, 12], [10, 13], [8, 14], [14, 15], [14, 16], [14, 17]],
    "bond_cut_atoms": [0]
  },
  "NMET": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "SD",    "CE",   "HE1",   "HE2",   "HE3",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",    "SA",     "C",     "H",     "H",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.20,    0.15,    0.00,    0.00,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    0.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00214,-0.00143, 0.00000, 0.00000, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,     2.5,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.0,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     2.0,     1.9,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.272,  -0.273,   0.266,   0.000,  -0.353,   0.084,   0.000,   0.000,   0.075,   0.000,   0.000,  -0.168,   0.083,   0.000,   0.000,   0.000,   0.338,   0.338,   0.338],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 16], [4, 17], [4, 18], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [12, 13], [12, 14], [12, 15]],
    "bond_cut_atoms": [0]
  },
  "NASN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "ND2",  "HD22",  "HD21",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,     1.9,    null,    true,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,     5.0,    null,    true,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.276,   0.000,  -0.352,   0.173,   0.000,   0.000,   0.220,  -0.279,  -0.374,   0.157,   0.157,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 13], [4, 14], [4, 15], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [10, 11], [10, 12]],
    "bond_cut_atoms": [0]
  },
  "NGLN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "NE2",  "HE22",  "HE21",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    null,    true,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    null,    true,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.272,   0.267,   0.000,  -0.352,   0.084,   0.000,   0.000,   0.105,   0.000,   0.000,   0.214,  -0.279,  -0.374,   0.157,   0.157,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 16], [4, 17], [4, 18], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [13, 14], [13, 15]],
    "bond_cut_atoms": [0]
  },
  "NARG": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "NE",    "HE",    "CZ",   "NH1",  "HH12",  "HH11",   "NH2",  "HH22",  "HH21",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",     "C",     "N",    "HD",    "HD",     "N",    "HD",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.15,    0.16,    0.02,    0.02,    0.16,    0.02,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    4.00,    3.50,    2.00,    2.00,    3.50,    2.00,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000, 33.5103, 22.4493,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051,-0.00143,-0.00162, 0.00051, 0.00051,-0.00162, 0.00051, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,     1.9,     1.8,    null,    null,     1.8,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.271,   0.268,   0.000,  -0.349,   0.078,   0.000,   0.000,   0.044,   0.000,   0.000,   0.195,   0.000,   0.000,  -0.281,   0.257,   0.335,  -0.294,   0.253,   0.253,  -0.294,   0.253,   0.253,   0.342,   0.342,   0.342],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 23], [4, 24], [4, 25], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [11, 14], [14, 15], [14, 16], [16, 17], [17, 18], [17, 19], [16, 20], [20, 21], [20, 22]],
    "bond_cut_atoms": [0]
  },
  "NSER": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "OG",    "HG",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",    "OA",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.20,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    3.20,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 17.1573,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00251, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,     1.9,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,     5.0,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.7,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.276,  -0.272,   0.291,   0.000,  -0.351,   0.227,   0.000,   0.000,  -0.395,   0.207,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 10], [4, 11], [4, 12], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9]],
    "bond_cut_atoms": [0]
  },
  "NTHR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "OG1",   "HG1",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "C",     "H",     "H",     "H",    "OA",    "HD",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20,    0.02,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20,    2.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251, 0.00051, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,     1.9,    null,    null,    null,     1.7,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.275,  -0.272,   0.294,   0.000,  -0.350,   0.176,   0.000,   0.044,   0.000,   0.000,   0.000,  -0.392,   0.208,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 13], [4, 14], [4, 15], [2, 5], [5, 6], [5, 7], [7, 8], [7, 9], [7, 10], [5, 11], [11, 12]],
    "bond_cut_atoms": [0]
  },
  "NVAL": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",    "HB",   "CG1",  "HG11",  "HG12",  "HG13",   "CG2",  "HG21",  "HG22",  "HG23",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.272,  -0.274,   0.267,   0.000,  -0.354,   0.050,   0.000,   0.014,   0.000,   0.000,   0.000,   0.014,   0.000,   0.000,   0.000,   0.337,   0.337,   0.337],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 15], [4, 16], [4, 17], [2, 5], [5, 6], [5, 7], [7, 8], [7, 9], [7, 10], [5, 11], [11, 12], [11, 13], [11, 14]],
    "bond_cut_atoms": [0]
  },
  "NTRP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "NE1",   "HE1",   "CE2",   "CD2",   "CE3",   "HE3",   "CZ3",   "HZ3",   "CH2",   "HH2",   "CZ2",   "HZ2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",     "A",     "H",     "N",    "HD",     "A",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.16,    0.02,    0.15,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    3.50,    2.00,    4.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.9,    null,     1.8,    null,     1.9,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.273,  -0.272,   0.270,   0.000,  -0.353,   0.111,   0.000,   0.000,  -0.016,   0.086,   0.000,  -0.366,   0.163,   0.045,   0.003,   0.010,   0.000,   0.001,   0.000,   0.002,   0.000,   0.026,   0.000,   0.339,   0.339,   0.339],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 23], [4, 24], [4, 25], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [9, 11], [11, 12], [11, 13], [13, 14], [14, 15], [15, 16], [15, 17], [17, 18], [17, 19], [19, 20], [19, 21], [21, 22], [14, 8], [21, 13]],
    "bond_cut_atoms": [0]
  },
  "NTYR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "OH",    "HH",   "CE2",   "HE2",   "CD2",   "HD2",    "H1",    "H2",    "H3"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",    "OA",    "HD",     "A",     "H",     "A",     "H",    "HD",    "HD",    "HD"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.20,    0.02,    0.15,    0.00,    0.15,    0.00,    0.02,    0.02,    0.02],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    3.20,    2.00,    4.00,    0.00,    4.00,    0.00,    2.00,    2.00,    2.00],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103, 17.1573,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000,  0.0000],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052,-0.00251, 0.00051,-0.00052, 0.00000,-0.00052, 0.00000, 0.00051, 0.00051, 0.00051],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    null,    null,    null,    null,    true,    true,    true],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    null,    null,    null,    null,    true,    true,    true],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,     1.7,    null,     1.9,    null,     1.9,    null,    null,    null,    null],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.274,  -0.272,   0.271,   0.000,  -0.352,   0.109,   0.000,   0.000,  -0.042,   0.008,   0.000,   0.045,   0.000,   0.113,  -0.516,   0.289,   0.045,   0.000,   0.008,   0.000,   0.340,   0.340,   0.340],
             "bonds": [[0, 1], [0, 2], [2, 3], [2, 4], [4, 20], [4, 21], [4, 22], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [9, 10], [9, 11], [11, 12], [11, 13], [13, 14], [14, 15], [13, 16], [16, 17], [16, 18], [18, 19], [18, 8]],
    "bond_cut_atoms": [0]
  },
  "CALA": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB1",   "HB2",   "HB3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.531,   0.129,   0.000,  -0.339,   0.169,   0.038,   0.000,   0.000,   0.000,  -0.532],
             "bonds": [[10, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [4]
  },
  "CCYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG",    "HG",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.067,  -0.534,   0.140,   0.000,  -0.338,   0.168,   0.099,   0.000,   0.000,  -0.173,   0.105,  -0.534],
             "bonds": [[11, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10]],
    "bond_cut_atoms": [4]
  },
  "CCYX": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.067,  -0.533,   0.143,   0.000,  -0.338,   0.168,   0.115,   0.000,   0.000,  -0.089,  -0.533],
             "bonds": [[10, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [4, 9]
  },
  "CCYM": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "SG",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "SA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00214,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     2.5,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     1.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     2.0,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.063,  -0.537,   0.106,   0.000,  -0.343,   0.167,  -0.145,   0.000,   0.000,  -0.774,  -0.537],
             "bonds": [[10, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9]],
    "bond_cut_atoms": [4]
  },
  "CASP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",    "OA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,     1.9,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,     5.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.135,   0.000,  -0.342,   0.167,   0.085,   0.000,   0.000,   0.045,  -0.539,  -0.540,  -0.538],
             "bonds": [[12, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11]],
    "bond_cut_atoms": [4]
  },
  "CASH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "OD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,     1.9,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,     5.0,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.141,   0.000,  -0.342,   0.167,   0.156,   0.000,   0.000,   0.310,  -0.248,  -0.473,   0.301,  -0.539],
             "bonds": [[13, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12]],
    "bond_cut_atoms": [4]
  },
  "CGLU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.130,   0.000,  -0.342,   0.167,   0.038,   0.000,   0.000,   0.056,   0.000,   0.000,   0.042,  -0.540,  -0.540,  -0.538],
             "bonds": [[15, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14]],
    "bond_cut_atoms": [4]
  },
  "CGLH": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "OE2",   "HE2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",    "OA",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.20,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.20,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 17.1573,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00251, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,     1.9,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,     5.0,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.7,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,    true,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.540,   0.130,   0.000,  -0.342,   0.167,   0.044,   0.000,   0.000,   0.127,   0.000,   0.000,   0.308,  -0.248,  -0.473,   0.301,  -0.539],
             "bonds": [[16, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [14, 15]],
    "bond_cut_atoms": [4]
  },
  "CPHE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "HZ",   "CE2",   "HE2",   "CD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false,    true,   false,   false],
         "gasteiger": [   0.066,  -0.532,   0.136,   0.000,  -0.338,   0.169,   0.068,   0.000,   0.000,  -0.045,   0.004,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.000,   0.004,   0.000,  -0.532],
             "bonds": [[20, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [14, 16], [16, 17], [16, 18], [18, 19], [18, 9]],
    "bond_cut_atoms": [4]
  },
  "CGLY": {
        "atom_names": [     "C",     "O",    "CA",   "HA2",     "N",     "H",   "HA3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.063,  -0.531,   0.173,   0.000,  -0.341,   0.168,   0.000,  -0.532],
             "bonds": [[7, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6]],
    "bond_cut_atoms": [4]
  },
  "CHIE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",    "NA",     "A",     "H",     "N",    "HD",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,    null,    null,    null,    true,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,    null,    null,    null,    true,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,     1.9,    null,     1.8,    null,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.537,   0.136,   0.000,  -0.342,   0.167,   0.091,   0.000,   0.000,   0.062,  -0.238,   0.200,   0.000,  -0.345,   0.170,   0.109,   0.000,  -0.538],
             "bonds": [[17, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [11, 12], [11, 13], [13, 14], [13, 15], [15, 16], [15, 9]],
    "bond_cut_atoms": [4]
  },
  "CHID": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "CD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",    "NA",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,     1.9,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,     5.0,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.537,   0.136,   0.000,  -0.342,   0.167,   0.088,   0.000,   0.000,   0.038,  -0.342,   0.170,   0.200,   0.000,  -0.241,   0.136,   0.000,  -0.538],
             "bonds": [[17, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [15, 9]],
    "bond_cut_atoms": [4]
  },
  "CHIP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "ND1",   "HD1",   "CE1",   "HE1",   "NE2",   "HE2",   "CD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "N",    "HD",     "A",     "H",     "N",    "HD",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.16,    0.02,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.50,    2.00,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 22.4493,  0.0000, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00162, 0.00051,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    true,    null,    null,    null,    true,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.8,    null,     1.9,    null,     1.8,    null,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.541,   0.138,   0.000,  -0.343,   0.166,   0.117,   0.000,   0.000,   0.144,  -0.244,   0.316,   0.405,   0.000,  -0.246,   0.316,   0.248,   0.000,  -0.541],
             "bonds": [[18, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [14, 16], [16, 17], [16, 9]],
    "bond_cut_atoms": [4]
  },
  "CILE": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "CG1",  "HG12",  "HG13",   "CD1",  "HD11",  "HD12",  "HD13",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,     1.9,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.531,   0.135,   0.000,  -0.338,   0.169,   0.014,   0.000,   0.010,   0.000,   0.000,   0.000,   0.002,   0.000,   0.000,   0.004,   0.000,   0.000,   0.000,  -0.531],
             "bonds": [[19, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18]],
    "bond_cut_atoms": [4]
  },
  "CLYS": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ1",   "HZ2",   "HZ3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",    "HD",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    true,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.130,   0.000,  -0.343,   0.167,   0.031,   0.000,   0.000,   0.008,   0.000,   0.000,   0.062,   0.000,   0.000,   0.262,   0.000,   0.000,  -0.352,   0.349,   0.349,   0.349,  -0.539],
             "bonds": [[22, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18], [18, 19], [18, 20], [18, 21]],
    "bond_cut_atoms": [4]
  },
  "CLYN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "CE",   "HE2",   "HE3",    "NZ",   "HZ2",   "HZ3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",    "NA",    "HD",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.537,   0.131,   0.000,  -0.340,   0.168,   0.032,   0.000,   0.000,   0.004,   0.000,   0.000,   0.015,   0.000,   0.000,   0.079,   0.000,   0.000,  -0.323,   0.121,   0.121,  -0.536],
             "bonds": [[21, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [15, 18], [18, 19], [18, 20]],
    "bond_cut_atoms": [4]
  },
  "CLEU": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",    "HG",   "CD1",  "HD11",  "HD12",  "HD13",   "CD2",  "HD21",  "HD22",  "HD23",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.531,   0.132,   0.000,  -0.338,   0.169,   0.035,   0.000,   0.000,  -0.015,   0.000,   0.007,   0.000,   0.000,   0.000,   0.007,   0.000,   0.000,   0.000,  -0.532],
             "bonds": [[19, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12], [11, 13], [11, 14], [9, 15], [15, 16], [15, 17], [15, 18]],
    "bond_cut_atoms": [4]
  },
  "CMET": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "SD",    "CE",   "HE1",   "HE2",   "HE3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",    "SA",     "C",     "H",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.20,    0.15,    0.00,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00214,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     2.5,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     1.0,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     2.0,     1.9,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.535,   0.132,   0.000,  -0.339,   0.168,   0.043,   0.000,   0.000,   0.074,   0.000,   0.000,  -0.161,   0.087,   0.000,   0.000,   0.000,  -0.535],
             "bonds": [[17, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [13, 14], [13, 15], [13, 16]],
    "bond_cut_atoms": [4]
  },
  "CASN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "OD1",   "ND2",  "HD22",  "HD21",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,     1.9,    null,    true,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,     5.0,    null,    true,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.140,   0.000,  -0.342,   0.167,   0.132,   0.000,   0.000,   0.223,  -0.270,  -0.363,   0.162,   0.162,  -0.538],
             "bonds": [[14, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [11, 12], [11, 13]],
    "bond_cut_atoms": [4]
  },
  "CGLN": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "HG2",   "HG3",    "CD",   "OE1",   "NE2",  "HE22",  "HE21",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",    "OA",     "N",    "HD",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.20,    0.16,    0.02,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    3.20,    3.50,    2.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 17.1573, 22.4493,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143,-0.00251,-0.00162, 0.00051, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     1.9,    null,    true,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,     5.0,    null,    true,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,     1.7,     1.8,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.539,   0.130,   0.000,  -0.342,   0.167,   0.042,   0.000,   0.000,   0.104,   0.000,   0.000,   0.221,  -0.270,  -0.363,   0.162,   0.162,  -0.539],
             "bonds": [[17, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [14, 15], [14, 16]],
    "bond_cut_atoms": [4]
  },
  "CARG": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",    "NE",    "HE",    "CZ",   "NH1",  "HH12",  "HH11",   "NH2",  "HH22",  "HH21",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",     "N",    "HD",     "C",     "N",    "HD",    "HD",     "N",    "HD",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.16,    0.02,    0.15,    0.16,    0.02,    0.02,    0.16,    0.02,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.50,    2.00,    4.00,    3.50,    2.00,    2.00,    3.50,    2.00,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 22.4493,  0.0000, 33.5103, 22.4493,  0.0000,  0.0000, 22.4493,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00162, 0.00051,-0.00143,-0.00162, 0.00051, 0.00051,-0.00162, 0.00051, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    true,    true,    null,    true,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.8,    null,     1.9,     1.8,    null,    null,     1.8,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,    true,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.065,  -0.541,   0.130,   0.000,  -0.344,   0.166,   0.034,   0.000,   0.000,   0.040,   0.000,   0.000,   0.199,   0.000,   0.000,  -0.275,   0.263,   0.342,  -0.287,   0.259,   0.259,  -0.287,   0.259,   0.259,  -0.541],
             "bonds": [[24, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [9, 11], [9, 12], [12, 13], [12, 14], [12, 15], [15, 16], [15, 17], [17, 18], [18, 19], [18, 20], [17, 21], [21, 22], [21, 23]],
    "bond_cut_atoms": [4]
  },
  "CSER": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "OG",    "HG",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",    "OA",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.20,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    3.20,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 17.1573,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00251, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,     1.9,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,     5.0,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.7,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.067,  -0.535,   0.156,   0.000,  -0.339,   0.168,   0.191,   0.000,   0.000,  -0.386,   0.214,  -0.536],
             "bonds": [[11, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10]],
    "bond_cut_atoms": [4]
  },
  "CTHR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG2",  "HG21",  "HG22",  "HG23",   "OG1",   "HG1",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",    "OA",    "HD",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20,    0.02,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20,    2.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251, 0.00051,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     1.9,    true,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,     5.0,    true,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.7,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.067,  -0.536,   0.159,   0.000,  -0.339,   0.168,   0.142,   0.000,   0.042,   0.000,   0.000,   0.000,  -0.383,   0.216,  -0.536],
             "bonds": [[14, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13]],
    "bond_cut_atoms": [4]
  },
  "CVAL": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",    "HB",   "CG1",  "HG11",  "HG12",  "HG13",   "CG2",  "HG21",  "HG22",  "HG23",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "C",     "H",     "H",     "H",     "C",     "H",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.15,    0.00,    0.00,    0.00,    0.15,    0.00,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    4.00,    0.00,    0.00,    0.00,    4.00,    0.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,     1.9,    null,    null,    null,     1.9,    null,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,    true,   false,   false,   false,    true,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.531,   0.134,   0.000,  -0.338,   0.169,   0.011,   0.000,   0.010,   0.000,   0.000,   0.000,   0.010,   0.000,   0.000,   0.000,  -0.531],
             "bonds": [[16, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [8, 9], [8, 10], [8, 11], [6, 12], [12, 13], [12, 14], [12, 15]],
    "bond_cut_atoms": [4]
  },
  "CTRP": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "NE1",   "HE1",   "CE2",   "CD2",   "CE3",   "HE3",   "CZ3",   "HZ3",   "CH2",   "HH2",   "CZ2",   "HZ2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "N",    "HD",     "A",     "A",     "A",     "H",     "A",     "H",     "A",     "H",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.16,    0.02,    0.15,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    3.50,    2.00,    4.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00162, 0.00051,-0.00052,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.8,    null,     1.9,     1.9,     1.9,    null,     1.9,    null,     1.9,    null,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,    true,    true,   false,    true,   false,    true,   false,    true,   false,   false],
         "gasteiger": [   0.065,  -0.536,   0.135,   0.000,  -0.341,   0.167,   0.070,   0.000,   0.000,  -0.020,   0.089,   0.000,  -0.353,   0.170,   0.047,   0.003,   0.010,   0.000,   0.001,   0.000,   0.002,   0.000,   0.027,   0.000,  -0.536],
             "bonds": [[24, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [16, 17], [16, 18], [18, 19], [18, 20], [20, 21], [20, 22], [22, 23], [15, 9], [22, 14]],
    "bond_cut_atoms": [4]
  },
  "CTYR": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",     "H",    "CB",   "HB2",   "HB3",    "CG",   "CD1",   "HD1",   "CE1",   "HE1",    "CZ",    "OH",    "HH",   "CE2",   "HE2",   "CD2",   "HD2",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",    "HD",     "C",     "H",     "H",     "A",     "A",     "H",     "A",     "H",     "A",    "OA",    "HD",     "A",     "H",     "A",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.02,    0.15,    0.00,    0.00,    0.15,    0.15,    0.00,    0.15,    0.00,    0.15,    0.20,    0.02,    0.15,    0.00,    0.15,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    2.00,    4.00,    0.00,    0.00,    4.00,    4.00,    0.00,    4.00,    0.00,    4.00,    3.20,    2.00,    4.00,    0.00,    4.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103, 33.5103,  0.0000, 33.5103,  0.0000, 33.5103, 17.1573,  0.0000, 33.5103,  0.0000, 33.5103,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162, 0.00051,-0.00143, 0.00000, 0.00000,-0.00052,-0.00052, 0.00000,-0.00052, 0.00000,-0.00052,-0.00251, 0.00051,-0.00052, 0.00000,-0.00052, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9,    true,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    true,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0,    true,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,    null,     1.9,    null,    null,     1.9,     1.9,    null,     1.9,    null,     1.9,     1.7,    null,     1.9,    null,     1.9,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,   false,    true,   false,   false,    true,    true,   false,    true,   false,   false,   false,   false,    true,   false,    true,   false,   false],
         "gasteiger": [   0.065,  -0.538,   0.135,   0.000,  -0.341,   0.167,   0.067,   0.000,   0.000,  -0.044,   0.007,   0.000,   0.047,   0.000,   0.117,  -0.498,   0.299,   0.047,   0.000,   0.007,   0.000,  -0.537],
             "bonds": [[21, 0], [0, 1], [0, 2], [2, 3], [2, 4], [4, 5], [2, 6], [6, 7], [6, 8], [6, 9], [9, 10], [10, 11], [10, 12], [12, 13], [12, 14], [14, 15], [15, 16], [14, 17], [17, 18], [17, 19], [19, 20], [19, 9]],
    "bond_cut_atoms": [4]
  },
  "CPRO": {
        "atom_names": [     "C",     "O",    "CA",    "HA",     "N",    "CB",   "HB3",   "HB2",    "CG",   "HG2",   "HG3",    "CD",   "HD2",   "HD3",   "OXT"],
        "atom_types": [     "C",    "OA",     "C",     "H",     "N",     "C",     "H",     "H",     "C",     "H",     "H",     "C",     "H",     "H",    "OA"],
         "ad4_epsii": [    0.15,    0.20,    0.15,    0.00,    0.16,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.15,    0.00,    0.00,    0.20],
           "ad4_rii": [    4.00,    3.20,    4.00,    0.00,    3.50,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    4.00,    0.00,    0.00,    3.20],
       "ad4_sol_vol": [ 33.5103, 17.1573, 33.5103,  0.0000, 22.4493, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 33.5103,  0.0000,  0.0000, 17.1573],
       "ad4_sol_par": [-0.00143,-0.00251,-0.00143, 0.00000,-0.00162,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00143, 0.00000, 0.00000,-0.00251],
        "ad4_hb_rij": [    null,     1.9,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     1.9],
      "ad4_hb_epsij": [    null,     5.0,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,    null,     5.0],
           "vina_ri": [     1.9,     1.7,     1.9,    null,     1.8,     1.9,    null,    null,     1.9,    null,    null,     1.9,    null,    null,     1.7],
        "vina_donor": [   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false],
     "vina_acceptor": [   false,    true,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,   false,    true],
  "vina_hydrophobic": [   false,   false,   false,   false,   false,    true,   false,   false,    true,   false,   false,   false,   false,   false,   false],
         "gasteiger": [   0.066,  -0.530,   0.135,   0.000,  -0.324,   0.034,   0.000,   0.000,   0.025,   0.000,   0.000,   0.124,   0.000,   0.000,  -0.530],
             "bonds": [[14, 0], [0, 1], [0, 2], [2, 3], [2, 4], [2, 5], [5, 6], [5, 7], [5, 8], [8, 9], [8, 10], [8, 11], [11, 12], [11, 13], [11, 4]],
    "bond_cut_atoms": [4]
  }
}
"""
flex_residue_json = """{
"LYS":
    {"is_atom": [false, true, false, false, true, false, true, false, true, false, true, false, true, true, true, true, false, false, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", null, "CD", null, "CE", null, "NZ", "HZ3", "HZ2", "HZ1", null, null, null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, "BRANCH   3   4", null, "BRANCH   4   5", null, "BRANCH   5   6", null, null, null, null, "ENDBRANCH   5   6", "ENDBRANCH   4   5", "ENDBRANCH   3   4", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"ARG":
    {"is_atom": [false, true, false, false, true, false, true, false, true, false, true, true, true, true, true, true, true, true, true, false, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", null, "CD", null, "NE", "CZ", "NH2", "NH1", "HH22", "HH21", "HH12", "HH11", "HE", null, null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, "BRANCH   3   4", null, "BRANCH   4   5", null, null, null, null, null, null, null, null, null, "ENDBRANCH   4   5", "ENDBRANCH   3   4", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"GLH":
    {"is_atom": [false, true, false, false, true, true, true, false, true, true, true, false, true, true, true, true, false, false, false], "atom_name": [null, "CA", null, null, "CB", "HB2", "HB3", null, "CG", "HG3", "HG2", null, "CD", "OE1", "OE2", "HE2", null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, null, null, "BRANCH   2   5", null, null, null, "BRANCH   5   8", null, null, null, null, "ENDBRANCH   5   8", "ENDBRANCH   2   5", "ENDBRANCH   1   2"]},

"HIP":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "ND1", "HD1", "CE1", "NE2", "CD2", "HE2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"VAL":
    {"is_atom": [false, true, false, false, true, true, true, false], "atom_name": [null, "CA", null, null, "CB", "CG1", "CG2", null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, null, null, "ENDBRANCH   1   2"]},

"TYR":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, false, true, true, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "CD1", "CE1", "CZ", "CE2", "CD2", null, "OH", "HH", null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, "BRANCH   6   9", null, null, "ENDBRANCH   6   9", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"ASH":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "OD1", "OD2", "HD2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"HIE":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "ND1", "CE1", "NE2", "HE2", "CD2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"GLU":
    {"is_atom": [false, true, false, false, true, false, true, false, true, true, true, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", null, "CD", "OE1", "OE2", null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, "BRANCH   3   4", null, null, null, "ENDBRANCH   3   4", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"LEU":
    {"is_atom": [false, true, false, false, true, false, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "CD2", "CD1", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"PHE":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "CD1", "CE1", "CZ", "CE2", "CD2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"GLN":
    {"is_atom": [false, true, false, false, true, false, true, false, true, true, true, true, true, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", null, "CD", "NE2", "HE22", "HE21", "OE1", null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, "BRANCH   3   4", null, null, null, null, null, "ENDBRANCH   3   4", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"ILE":
    {"is_atom": [false, true, false, false, true, true, false, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", "CG2", null, "CG1", "CD1", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, null, "BRANCH   2   4", null, null, "ENDBRANCH   2   4", "ENDBRANCH   1   2"]},

"MET":
    {"is_atom": [false, true, false, false, true, false, true, false, true, true, false, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", null, "SD", "CE", null, null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, "BRANCH   3   4", null, null, "ENDBRANCH   3   4", "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"ASN":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "OD1", "ND2", "HD21", "HD22", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"THR":
    {"is_atom": [false, true, false, false, true, true, false, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", "CG2", null, "OG1", "HG1", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, null, "BRANCH   2   4", null, null, "ENDBRANCH   2   4", "ENDBRANCH   1   2"]},

"TRP":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "CD2", "CE2", "CE3", "CZ2", "NE1", "CH2", "CZ3", "HE1", "CD1", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"CYS":
    {"is_atom": [false, true, false, false, true, false, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "SG", "HG", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"SER":
    {"is_atom": [false, true, false, false, true, false, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "OG", "HG", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"ASP":
    {"is_atom": [false, true, false, false, true, false, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "OD1", "OD2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]},

"HID":
    {"is_atom": [false, true, false, false, true, false, true, true, true, true, true, true, false, false], "atom_name": [null, "CA", null, null, "CB", null, "CG", "ND1", "HD1", "CE1", "NE2", "CD2", null, null], "original_line": ["ROOT", null, "ENDROOT", "BRANCH   1   2", null, "BRANCH   2   3", null, null, null, null, null, null, "ENDBRANCH   2   3", "ENDBRANCH   1   2"]}
}
"""

atom_property_definitions = {'H': 'vdw', 'C': 'vdw', 'A': 'vdw', 'N': 'vdw', 'P': 'vdw', 'S': 'vdw',
                             'Br': 'vdw', 'I': 'vdw', 'F': 'vdw', 'Cl': 'vdw',
                             'NA': 'hb_acc', 'OA': 'hb_acc', 'SA': 'hb_acc', 'OS': 'hb_acc', 'NS': 'hb_acc',
                             'HD': 'hb_don', 'HS': 'hb_don',
                             'Mg': 'metal', 'Ca': 'metal', 'Fe': 'metal', 'Zn': 'metal', 'Mn': 'metal',
                             'MG': 'metal', 'CA': 'metal', 'FE': 'metal', 'ZN': 'metal', 'MN': 'metal',
                             'W': 'water',
                             'G0': 'glue', 'G1': 'glue', 'G2': 'glue', 'G3': 'glue',
                             'CG0': 'glue', 'CG1': 'glue', 'CG2': 'glue', 'CG3': 'glue'}

covalent_radius = {
    "H":  0.31,
    "D":  0.00,
    "He": 0.28,
    "Li": 1.28,
    "Be": 0.96,
    "B":  0.84,
    "C":  0.76,
    "N":  0.71,
    "O":  0.66,
    "F":  0.57,
    "Ne": 0.58,
    "Na": 1.66,
    "Mg": 1.41,
    "Al": 1.21,
    "Si": 1.11,
    "P":  1.07,
    "S":  1.05,
    "Cl": 1.02,
    "Ar": 1.06,
    "K ": 2.03,
    "Ca": 1.76,
    "Sc": 1.70,
    "Ti": 1.60,
    "V":  1.53,
    "Cr": 1.39,
    "Mn": 1.39,
    "Fe": 1.32,
    "Co": 1.26,
    "Ni": 1.24,
    "Cu": 1.32,
    "Zn": 1.22,
    "Ga": 1.22,
    "Ge": 1.20,
    "As": 1.19,
    "Se": 1.20,
    "Br": 1.20,
    "Kr": 1.16,
    "Rb": 2.20,
    "Sr": 1.95,
    "Y":  1.90,
    "Zr": 1.75,
    "Nb": 1.64,
    "Mo": 1.54,
    "Tc": 1.47,
    "Ru": 1.46,
    "Rh": 1.42,
    "Pd": 1.39,
    "Ag": 1.45,
    "Cd": 1.44,
    "In": 1.42,
    "Sn": 1.39,
    "Sb": 1.39,
    "Te": 1.38,
    "I":  1.39,
    "Xe": 1.40,
    "Cs": 2.44,
    "Ba": 2.15,
    "La": 2.07,
    "Ce": 2.04,
    "Pr": 2.03,
    "Nd": 2.01,
    "Pm": 1.99,
    "Sm": 1.98,
    "Eu": 1.98,
    "Gd": 1.96,
    "Tb": 1.94,
    "Dy": 1.92,
    "Ho": 1.92,
    "Er": 1.89,
    "Tm": 1.90,
    "Yb": 1.87,
    "Lu": 1.87,
    "Hf": 1.75,
    "Ta": 1.70,
    "W":  1.62,
    "Re": 1.51,
    "Os": 1.44,
    "Ir": 1.41,
    "Pt": 1.36,
    "Au": 1.36,
    "Hg": 1.32,
    "Tl": 1.45,
    "Pb": 1.46,
    "Bi": 1.48,
    "Po": 1.40,
    "At": 1.50,
    "Rn": 1.50,
    "Fr": 2.60,
    "Ra": 2.21,
    "Ac": 2.15,
    "Th": 2.06,
    "Pa": 2.00,
    "U":  1.96,
    "Np": 1.90,
    "Pu": 1.87,
    "Am": 1.80,
    "Cm": 1.69,
    "Bk": 1.60,
    "Cf": 1.60,
    "Es": 1.60,
    "Fm": 1.60,
    "Md": 1.60,
    "No": 1.60,
    "Lr": 1.60,
    "Rf": 1.60,
    "Db": 1.60,
    "Sg": 1.60,
    "Bh": 1.60,
    "Hs": 1.60,
    "Mt": 1.60,
    "Ds": 1.60,
    "Rg": 1.60,
    "Cn": 1.60,
    "Nh": 1.60,
    "Fl": 1.60,
    "Mc": 1.60,
    "Lv": 1.60,
    "Ts": 1.60,
    "Og": 1.60
}

autodock4_atom_types_elements = {
    'H':   'H',
    'HD':  'H',
    'HS':  'H',
    'C':   'C',
    'A':   'C',
    'N':   'N',
    'NA':  'N',
    'NS':  'N',
    'OA':  'O',
    'OS':  'O',
    'F':   'F',
    'Mg':  'Mg',
    'MG':  'Mg',
    'P':   'P',
    'SA':  'S',
    'S':   'S',
    'Cl':  'Cl',
    'CL':  'Cl',
    'Ca':  'Ca',
    'CA':  'Ca',
    'Mn':  'Mn',
    'MN':  'Mn',
    'Fe':  'Fe',
    'FE':  'Fe',
    'Zn':  'Zn',
    'ZN':  'Zn',
    'Br':  'Br',
    'BR':  'Br',
    'I':   'I',
    'Si':  'Si',
    'At':  'At',
    'G0':  'C',
    'G1':  'C',
    'G2':  'C',
    'G3':  'C',
    'CG0': 'C',
    'CG1': 'C',
    'CG2': 'C',
    'CG3': 'C',
    'W':   'O'
}

residue_params = json.loads(residue_params_json)

flexres_templates = json.loads(flex_residue_json)

def _read_receptor_pdbqt_string(pdbqt_string, skip_typing=False):
    atoms = []
    atoms_dtype = [('idx', 'i4'), ('serial', 'i4'), ('name', 'U4'), ('resid', 'i4'),
                   ('resname', 'U3'), ('chain', 'U1'), ("xyz", "f4", (3)),
                   ('partial_charges', 'f4'), ('atom_type', 'U2'),
                   ('alt_id', 'U1'), ('in_code', 'U1'),
                   ('occupancy', 'f4'), ('temp_factor', 'f4'), ('record_type', 'U6')
                  ]
    atom_annotations = {'hb_acc': [], 'hb_don': [],
                        'all': [], 'vdw': [],
                        'metal': []}
    # TZ is a pseudo atom for AutoDock4Zn FF
    pseudo_atom_types = ['TZ']

    idx = 0
    for line in pdbqt_string.split(os_linesep):
        if line.startswith('ATOM') or line.startswith("HETATM"):
            serial = int(line[6:11].strip())
            name = line[12:16].strip()
            resname = line[17:20].strip()
            chainid = line[21].strip()
            resid = int(line[22:26].strip())
            xyz = np.array([line[30:38].strip(), line[38:46].strip(), line[46:54].strip()], dtype=np.float32)
            try:
                partial_charges = float(line[71:77].strip())
            except:
                partial_charges = None # probably reading a PDB, not PDBQT
            atom_type = line[77:79].strip()
            alt_id = line[16:17].strip()
            in_code = line[26:27].strip()
            try:
                occupancy = float(line[54:60])
            except:
                occupancy = None
            try:
                temp_factor = float(line[60:68])
            except:
                temp_factor = None
            record_type = line[0:6].strip()

            if skip_typing:
                atoms.append((idx, serial, name, resid, resname, chainid, xyz, partial_charges, atom_type,
                              alt_id, in_code, occupancy, temp_factor, record_type))
                continue
            if not atom_type in pseudo_atom_types:
                atom_annotations['all'].append(idx)
                atom_annotations[atom_property_definitions[atom_type]].append(idx)
                atoms.append((idx, serial, name, resid, resname, chainid, xyz, partial_charges, atom_type,
                              alt_id, in_code, occupancy, temp_factor, record_type))

            idx += 1

    atoms = np.array(atoms, dtype=atoms_dtype)

    return atoms, atom_annotations

def _write_pdbqt_line(atomidx, x, y, z, charge, atom_name, res_name, res_num, atom_type, chain,
                      alt_id=" ", in_code="", occupancy=1.0, temp_factor=0.0, record_type="ATOM"):
    if len(atom_name) > 4:
        raise ValueError("max length of atom_name is 4 but atom name is %s" % atom_name)
    atom_name = "%-3s" % atom_name
    line = "{:6s}{:5d} {:4s}{:1s}{:3s} {:1s}{:4d}{:1s}   {:8.3f}{:8.3f}{:8.3f}{:6.2f}{:6.2f}    {:6.3f} {:<2s}"
    line += os_linesep
    return line.format(record_type, atomidx, atom_name, alt_id, res_name, chain,
                   res_num, in_code, x, y, z,
                   occupancy, temp_factor, charge, atom_type)

class PDBQTReceptor:
    flexres_templates = flexres_templates
    skip_types=("H",)

    def __init__(self, pdbqt_filename, skip_typing=False):
        self._pdbqt_filename = pdbqt_filename
        self._atoms = None
        self._atom_annotations = None

        with open(pdbqt_filename) as f:
            pdbqt_string = f.read()

        self._atoms, self._atom_annotations = _read_receptor_pdbqt_string(pdbqt_string, skip_typing)
        self.atom_idxs_by_res = self.get_atom_indices_by_residue(self._atoms)

    def __repr__(self):
        return ('<Receptor from PDBQT file %s containing %d atoms>' % (self._pdbqt_filename, self._atoms.shape[0]))

    @staticmethod
    def get_atom_indices_by_residue(atoms):
        """ return a dictionary where residues are keys and
             values are lists of atom indices

            >>> atom_idx_by_res = {("A", "LYS", 417): [0, 1, 2, 3, ..., 8]}
        """

        atom_idx_by_res = {}
        for atom_index, atom in enumerate(atoms):
            res_id = (atom["chain"], atom["resname"], atom["resid"])
            atom_idx_by_res.setdefault(res_id, [])
            atom_idx_by_res[res_id].append(atom_index)
        return atom_idx_by_res

    @staticmethod
    def get_params_for_residue(resname, atom_names, residue_params=residue_params):
        excluded_params = ("atom_names", "bond_cut_atoms", "bonds")
        atom_params = {}
        atom_counter = 0
        err = ""
        ok = True
        is_matched = False
        for terminus in ["", "N", "C"]: # e.g. "CTYR" for C-term TYR, hard-coded in residue_params
            r_id = "%s%s" % (terminus, resname)
            if r_id not in residue_params:
                err = "residue %s not in residue_params" % r_id + os_linesep
                ok = False
                return atom_params, ok, err
            ref_names = set(residue_params[r_id]["atom_names"])
            query_names = set(atom_names)
            if ref_names == query_names:
                is_matched = True
                break

        if not is_matched:
            ok = False
            return atom_params, ok, err

        for atom_name in atom_names:
            name_index = residue_params[r_id]["atom_names"].index(atom_name)
            for param in residue_params[r_id].keys():
                if param in excluded_params:
                    continue
                if param not in atom_params:
                    atom_params[param] = [None] * atom_counter
                value = residue_params[r_id][param][name_index]
                atom_params[param].append(value)
            atom_counter += 1

        return atom_params, ok, err

    def assign_types_charges(self, residue_params=residue_params):
        wanted_params = ("atom_types", "gasteiger")
        atom_params = {key: [] for key in wanted_params}
        ok = True
        err = ""
        for r_id, atom_indices in self.atom_idxs_by_res.items():
            atom_names = tuple(self.atoms(atom_indices)["name"])
            resname = r_id[1]
            params_this_res, ok_, err_ = self.get_params_for_residue(resname, atom_names, residue_params)
            ok &= ok_
            err += err_
            if not ok_:
                print("did not match %s with template" % str(r_id), file=sys.stderr)
                continue
            for key in wanted_params:
                atom_params[key].extend(params_this_res[key])
        if ok:
            self._atoms["partial_charges"] = atom_params["gasteiger"]
            self._atoms["atom_type"] = atom_params["atom_types"]
        return ok, err

    def write_flexres_from_template(self, res_id, atom_index=0):
        success = True
        error_msg = ""
        branch_offset = atom_index # templates assume first atom is 1
        output = {"pdbqt": "", "flex_indices": [], "atom_index": atom_index}
        resname = res_id[1]
        if resname not in self.flexres_templates:
            success = False
            error_msg = "no flexible residue template for resname %s, sorry" % resname
            return output, success, error_msg
        if res_id not in self.atom_idxs_by_res:
            success = False
            chains = set(self._atoms["chain"])
            error_msg += "could not find residue with chain='%s', resname=%s, resnum=%d" % res_id + os_linesep
            error_msg += "chains in this receptor: %s" % ", ".join("'%s'" % c for c in chains) + os_linesep
            if " " in chains: # should not happen because we use strip() when parsing the chain
                error_msg += "use ' ' (a space character) for empty chain" + os_linesep
            if "" in chains:
                error_msg += "use '' (empty string) for empty chain" + os_linesep
            return output, success, error_msg

        # collect lines of res_id
        atoms_by_name = {}
        for i in self.atom_idxs_by_res[res_id]:
            name = self._atoms[i]["name"]
            if name in ['C', 'N', 'O', 'H', 'H1', 'H2', 'H3', 'OXT']: # skip backbone atoms
                continue
            atype = self._atoms[i]["atom_type"]
            if atype in self.skip_types:
                continue
            output["flex_indices"].append(i)
            atoms_by_name[name] = self.atoms(i)

        # check it was a full match
        template = self.flexres_templates[resname]
        got_atoms = set(atoms_by_name)
        ref_atoms = set()
        for i in range(len(template["is_atom"])):
            if template["is_atom"][i]:
                ref_atoms.add(template["atom_name"][i])
        if got_atoms != ref_atoms:
            success = False
            error_msg += "mismatch in atom names for residue %s" % str(res_id) + os_linesep
            error_msg += "names found but not in template: %s" % str(got_atoms.difference(ref_atoms)) + os_linesep
            error_msg += "missing names: %s" % str(ref_atoms.difference(got_atoms)) + os_linesep
            return output, success, error_msg

        # create output string
        n_lines = len(template['is_atom'])
        for i in range(n_lines):
            if template['is_atom'][i]:
                atom_index += 1
                name = template['atom_name'][i]
                atom = atoms_by_name[name]
                if atom["atom_type"] not in self.skip_types:
                    atom["serial"] = atom_index
                    output["pdbqt"] += self.write_pdbqt_line(atom)
            else:
                line = template['original_line'][i]
                if branch_offset > 0 and (line.startswith("BRANCH") or line.startswith("ENDBRANCH")):
                    keyword, i, j = line.split()
                    i = int(i) + branch_offset
                    j = int(j) + branch_offset
                    line = "%s %3d %3d" % (keyword, i, j)
                output["pdbqt"] += line + os_linesep # e.g. BRANCH keywords

        output["atom_index"] = atom_index
        return output, success, error_msg

    @staticmethod
    def write_pdbqt_line(atom):
        return _write_pdbqt_line(atom["serial"], atom["xyz"][0], atom["xyz"][1], atom["xyz"][2],
                                 atom["partial_charges"], atom["name"], atom["resname"],
                                 atom["resid"], atom["atom_type"], atom["chain"],
                                 atom["alt_id"], atom["in_code"], atom["occupancy"],
                                 atom["temp_factor"], atom["record_type"])

    def write_pdbqt_string(self, flexres=()):
        ok = True
        err = ""
        pdbqt = {"rigid": "",
                 "flex":  {},
                 "flex_indices": []}
        atom_index = 0
        for res_id in set(flexres):
            output, ok_, err_ = self.write_flexres_from_template(res_id, atom_index)
            atom_index = output["atom_index"] # next residue starts here
            ok &= ok_
            err += err_
            pdbqt["flex_indices"].extend(output["flex_indices"])
            pdbqt["flex"][res_id] = ""
            pdbqt["flex"][res_id] += "BEGIN_RES %3s %1s%4d" % (res_id) + os_linesep
            pdbqt["flex"][res_id] += output["pdbqt"]
            pdbqt["flex"][res_id] += "END_RES %3s %1s%4d" % (res_id) + os_linesep
        all_flex_pdbqt = ""
        for res_id, flexres_pdbqt in pdbqt["flex"].items():
            all_flex_pdbqt += flexres_pdbqt
        pdbqt['flex'] = all_flex_pdbqt

        # use non-flex lines for rigid part
        for i, atom in enumerate(self._atoms):
            if i not in pdbqt["flex_indices"] and atom["atom_type"] not in self.skip_types:
                pdbqt["rigid"] += self.write_pdbqt_line(atom)

        return pdbqt, ok, err

    def atoms(self, atom_idx=None):
        """Return the atom i

        Args:
            atom_idx (int, list): index of one or multiple atoms

        Returns:
            ndarray: 2d ndarray (atom_id, atom_name, resname, resid, chainid, xyz, q, t)

        """
        if atom_idx is not None and self._atoms.size > 1:
            if not isinstance(atom_idx, (list, tuple, np.ndarray)):
                atom_idx = np.array(atom_idx, dtype=int)
            atoms = self._atoms[atom_idx]
        else:
            atoms = self._atoms

        return atoms.copy()

class PDBQTEditor:
    def __init__(self, pdbqt_str: str | None=None):
        self.pdbqt_str = pdbqt_str
        self.pdbqt_chain_dict = {}

    def get_chain_and_aa_pos_aa(self, line):
        return line[21].strip(), int(line[22:26].strip()), line[17:20].strip()

    def parse_pdbqt_text_to_dict(self):
        self.pdbqt_chain_dict = {}
        chain_aa_dict = {}
        for line in self.pdbqt_str.split('\n'):
            if line.strip():
                chain, aa_pos, aa = self.get_chain_and_aa_pos_aa(line)
                if chain not in self.pdbqt_chain_dict:
                    self.pdbqt_chain_dict[chain] = {}
                    chain_aa_dict[chain] = []
                aa_pos_dict = self.pdbqt_chain_dict[chain]
                if aa_pos not in aa_pos_dict:
                    aa_pos_dict[aa_pos] = []
                    chain_aa_dict[chain].append(aa)
                aa_pos_dict[aa_pos].append(line)
        for chain, aa_pos_dict in self.pdbqt_chain_dict.items():
            aa_cnt = len(aa_pos_dict)
            for pos, text_list in aa_pos_dict.items():
                aa_pos_dict[pos] = '\n'.join(text_list)
            self.pdbqt_chain_dict[chain] = pd.DataFrame.from_dict(aa_pos_dict, 'index')
            self.pdbqt_chain_dict[chain]['Display'] = [True] * aa_cnt
            self.pdbqt_chain_dict[chain]['Flexible'] = [False] * aa_cnt
            self.pdbqt_chain_dict[chain]['AA_Name'] = chain_aa_dict[chain]

    def parse_logic(self, series: pd.Series, logic: str):
        def replace_expression(match):
            expr = match.group()
            negation = ''
            if expr.startswith('~'):
                negation = '~'
                expr = expr[1:]
            if '-' in expr:  # range
                start, end = map(int, expr.split('-'))
                return f"{negation}((series.index >= {start}) & (series.index <= {end}))"
            else:  # single value
                return f"{negation}(series.index == {expr})"

        logic = logic.replace(' ', '').replace(',', '|')    # "," is the same as or "|"
        logic_eval = re.sub(r'~?\d+-\d+|~?\d+(?:,~?\d+)*', replace_expression, logic)

        try:
            result = pd.eval(logic_eval)
            series[result] = True
        except:
            return None

        return series

    def update_display(self, chain: str, display_str: str | None):
        full_df = self.pdbqt_chain_dict[chain]
        if display_str is None:
            display_series = pd.Series([False] * len(full_df), list(full_df.index))
            full_df['Display'] = display_series
            return
        elif not display_str:
            display_series = pd.Series([True] * len(full_df), list(full_df.index))
            full_df['Display'] = display_series
            return
        display_series = pd.Series([False] * len(full_df), list(full_df.index)) # default to False
        result = self.parse_logic(display_series, display_str)
        if result is None:
            return f'Invalid syntax for chain {chain}.'
        full_df['Display'] = display_series

    def _condense_to_range(self, list_of_nums: list[int]):
        final_text = ''
        start = list_of_nums[0]
        end = list_of_nums[0]
        range_cnt = 1
        for num in list_of_nums[1:]:
            if num == start + range_cnt:
                end = num
                range_cnt += 1
            else:
                if start == end:
                    final_text += f'{start}|'
                else:
                    final_text += f'{start}-{end},'
                start = num
                end = num
                range_cnt = 1
        if start == end:
            final_text += f'{start}'
        else:
            final_text += f'{start}-{end}'
        return final_text

    def convert_to_range_text(self, chain: str):
        s = self.pdbqt_chain_dict[chain]['Display']
        mask = s == True
        displayed_pos = s[mask].index.to_list()
        if not displayed_pos:
            return f'~1-{max(s.index.to_list())}'
        if len(displayed_pos) == len(s):
            return ''   # empty text if everything is displayed
        return self._condense_to_range(displayed_pos)

    def convert_to_flex_set(self, get_pdbqt_str: bool=False):
        pdbqt_str = ''
        flex_res = set()
        for chain, df in self.pdbqt_chain_dict.items():
            display_mask = df['Display'] == True
            if get_pdbqt_str:
                pdbqt_str += '\n'.join(df[0][display_mask].to_list())
            flex_mask: pd.Series = (df['Flexible'] == True) & display_mask
            if not flex_mask.empty:
                flex_aa_names: pd.Series = df['AA_Name'][flex_mask]
                flex_res.update(set(zip([chain] * len(flex_aa_names), flex_aa_names, flex_aa_names.index)))
        if get_pdbqt_str:
            return flex_res, pdbqt_str
        return flex_res

    def convert_dict_to_pdbqt_text(self):
        pdbqt_str = ''
        for df in self.pdbqt_chain_dict.values():
            mask = df['Display'] == True
            pdbqt_str += '\n'.join(df[0][mask].to_list())
        return pdbqt_str

    def convert_to_flex_set(self, get_pdbqt_str: bool=False):
        pdbqt_str = ''
        flex_res = set()
        for chain, df in self.pdbqt_chain_dict.items():
            display_mask = df['Display'] == True
            if get_pdbqt_str:
                pdbqt_str += '\n'.join(df[0][display_mask].to_list())
                pdbqt_str += '\n'
            flex_mask: pd.Series = (df['Flexible'] == True) & display_mask
            if not flex_mask.empty:
                flex_aa_names: pd.Series = df['AA_Name'][flex_mask]
                flex_res.update(set(zip([chain] * len(flex_aa_names), flex_aa_names, flex_aa_names.index)))
        if get_pdbqt_str:
            return flex_res, pdbqt_str
        return flex_res

    def process_and_save_rigid_flex_processed_pdbqt(self, out_rigid_pth: str, out_flex_pth: str, out_processed_pth: str):
        flex_res, pdbqt_str = self.convert_to_flex_set(True)
        with open(out_processed_pth, 'w') as f:
            f.write(pdbqt_str)
        process_rigid_flex(pdbqt_str, out_rigid_pth, out_flex_pth, flex_res)
        return bool(flex_res)

    def check_format_type(self):
        for chain_dict in self.pdbqt_chain_dict.values():
            line = chain_dict.iloc[0, 0]
            if line[70:76].strip():
                return 'pdbqt'
            else:
                return 'pdb'

def process_rigid_flex(pdbqt_str: str, rigid_out_pth: str, flex_out_pth: str, flex_res: set):
    with tempfile.NamedTemporaryFile('w+', delete=False) as f:
        f.write(pdbqt_str)
        file = f.name
    receptor = PDBQTReceptor(file, skip_typing=True)
    pdbqt_string, _, _ = receptor.write_pdbqt_string(flex_res)
    with open(rigid_out_pth, 'w') as f:
        f.write(pdbqt_string['rigid'])
    if flex_res:
        with open(flex_out_pth, 'w') as f:
            f.write(pdbqt_string['flex'])

working_dir = os.path.join(os.path.basename(''), 'working_dir')
os.makedirs(working_dir, exist_ok=True)
float_num_regex = re.compile('[+-]?([0-9]*[.])?[0-9]+')
num_regex = re.compile('[0-9]+')

with open('dock_energy.csv', 'w') as f:
    f.write('name,energy\n')

def recursive_get_files(parent_dir: str):
    result = []
    for f in os.listdir(parent_dir):
        p = os.path.join(parent_dir, f)
        if os.path.isdir(p):
            result.extend(recursive_rm_file(p))
        else:
            result.append(p)
    return result

def recursive_rm_file(parent_dir: str):
    for f in os.listdir(parent_dir):
        p = os.path.join(parent_dir, f)
        if os.path.isdir(p):
            recursive_rm_file(p)
        elif f.startswith('.'):
            os.remove(p)

def convert_row_to_string_float(csv_row_str: str):
    name, score = csv_row_str.strip().split(',')
    if not score:
        score = float('nan')
    else:
        score = float(score)
    return name, score

class ProteinDockerInterface:
    def __init__(self):
        self.is_docking = False
        self.pdbqt_editor = None
        self.eng_regex = re.compile(r'REMARK VINA RESULT:\s+(-?\d+\.\d+)')
        self.dock_center = {'x': 0, 'y': 0, 'z': 0}
        self.box_width   = {'x': 0, 'y': 0, 'z': 0}
        self.param_dict = {'Exhaustiveness': 8,
                           'Poses'         : 10,
                           'Concurrent'    : 2,}
        self.display_flex_dict = {'Display' : 'A:1-15,20-35 B:15-20',
                                  'Flexible': 'A:10,A:25,B:17'}

    def start_interface(self):
        with gr.Blocks(css='footer{display:none !important}') as Interface:
            gr.Markdown('<span style="font-size:25px; font-weight:bold; ">AutoDock VINA Server Interface</span>')
            with gr.Row():
                with gr.Column(scale=2):
                    protein_stat = gr.Textbox(label='Protein Status :')
                    protein_input = gr.File(file_count='single',
                                            file_types=['.mds'],
                                            label='Protein File (mds)',)
                with gr.Column(scale=2):
                    ligand_stat = gr.Textbox(label='Ligand Status :')
                    ligand_input = gr.File(label='Ligand Files',
                                           file_count='multiple',
                                           file_types=['.zip', '.pdbqt'])
                with gr.Column(scale=1):
                    docked_stat = gr.Textbox(label='Docked Ligands :')
                    docked_input = gr.File(label='Docked Files',
                                           file_count='multiple',
                                           file_types=['.zip', '.pdbqt'])
            with gr.Row():
                with gr.Row():
                    with gr.Column(scale=1):
                        gr.Markdown('### Center Coordinate')
                        for v in self.dock_center:
                            self.dock_center[v] = gr.Number(label=v.lower(),
                                                            value=0,
                                                            interactive=True)
                    with gr.Column(scale=1):
                        gr.Markdown('### Box Size')
                        for v in self.box_width:
                            self.box_width[v] = gr.Number(label=v,
                                                          value=15,
                                                          interactive=True)
                    with gr.Column(scale=1):
                        gr.Markdown('### Parameters')
                        for p, default in self.param_dict.items():
                            if p == 'Exhaustiveness':
                                label = p + f' ({multiprocessing.cpu_count()})'
                            else:
                                label = p
                            self.param_dict[p] = gr.Number(label=label,
                                                           value=default,
                                                           interactive=True)
                    with gr.Column(scale=2):
                        gr.Markdown('### Protein Settings')
                        for p, placeholder in self.display_flex_dict.items():
                            self.display_flex_dict[p] = gr.Textbox(label=p,
                                                                   placeholder=placeholder,
                                                                   interactive=True)
                    with gr.Column(scale=1):
                        gr.Markdown('### VINA Type')
                        vina_selection = gr.Dropdown(['gnina'],
                                                     value='gnina')
            with gr.Row():
                dock_progress = gr.Text(label='Progress', interactive=False, scale=4)
                with gr.Column():
                    dock_button = gr.Button('Dock', interactive=False)
                    zip_checkbox = gr.Checkbox(label='Zip Result')
                    zip_progress = gr.Textbox(label='Progress', interactive=False)
                    zip_name = gr.Textbox(label='Zip File Name',
                                          placeholder='docked_result',
                                          interactive=True)
                    zipped_file_output = gr.File(label='Zipped file',
                                                 file_count='single',
                                                 file_types=['.zip'],)
            docking_status = gr.Textbox(label='Docking Status')
            energy_df = self.new_dataframe()
            protein_input.change(self.process_protein_file,
                                 inputs=protein_input,
                                 outputs=[protein_stat] + list(self.dock_center.values()) + \
                                     list(self.box_width.values()) + list(self.param_dict.values()) + \
                                         list(self.display_flex_dict.values()) + [dock_button])
            ligand_input.change(self.process_ligand,
                                inputs=ligand_input,
                                outputs=ligand_stat)
            docked_input.change(self.upload_docked_ligand,
                                inputs=docked_input,
                                outputs=docked_stat)
            dock_button.click(self.start_docking,
                              inputs=[vina_selection] + list(self.dock_center.values()) + \
                                  list(self.box_width.values()) + list(self.param_dict.values()) + \
                                      list(self.display_flex_dict.values()),
                              outputs=dock_progress)
            Interface.load(self.zip_docked_files, [zip_checkbox, zip_name],
                           [zipped_file_output, zip_checkbox, zip_progress], every=1)
            Interface.load(self.new_dataframe, None, energy_df, every=1)
            Interface.load(self.check_docking_status, None, docking_status, every=1)

            Interface.queue().launch(server_name="0.0.0.0", share=True, debug=True)

    def process_protein_file(self, file):
        update_dict = {'dock_center'   : [gr.update(), gr.update(), gr.update()],
                       'dock_width'    : [gr.update(), gr.update(), gr.update()],
                       'exhaustiveness': [gr.update()],
                       'eval_poses'    : [gr.update()],
                       'concurrent'    : [gr.update()],
                       'display'       : [gr.update()],
                       'flexible'      : [gr.update()],
                       'dock_status'   : [gr.update(interactive=False)]}
        self.pdbqt_editor = None
        if file:
            base_name = os.path.basename(file)
            docked_dir = os.path.join(working_dir, 'docked_dir')
            if not os.path.isdir(docked_dir):
                os.mkdir(docked_dir)
            cache_pth = os.path.join(docked_dir, 'cache_files')
            if not os.path.isdir(cache_pth):
                os.mkdir(cache_pth)
            if base_name.endswith('.pdbqt'):
                with open(file, 'r') as f:
                    pdbqt_str = f.readline()
                self.pdbqt_editor = PDBQTEditor(pdbqt_str)
                self.pdbqt_editor.parse_pdbqt_text_to_dict()
                text = 'Protein loaded'
            if base_name.endswith('.mds'):
                try:
                    with open(file, 'rb') as f:
                        mds_dict = pickle.load(f)
                    for k, v in mds_dict.items():
                        if k == 'dock_center':
                            update_dict['dock_center'] = [gr.update(value=v['x']),
                                                                    gr.update(value=v['y']),
                                                                    gr.update(value=v['z']),]
                        elif k == 'dock_width':
                            update_dict['dock_width'] = [gr.update(value=v['x']),
                                                                    gr.update(value=v['y']),
                                                                    gr.update(value=v['z']),]
                        elif k == 'pdbqt_editor':
                            self.pdbqt_editor = PDBQTEditor()
                            self.pdbqt_editor.pdbqt_chain_dict = v
                            display_strs = []
                            for chain in v:
                                r = self.pdbqt_editor.convert_to_range_text(chain)
                                if r:
                                    display_strs.append(f'{chain}:{r}')
                            update_dict['display'] = [gr.update(value=' '.join(display_strs))]
                            flex_set = self.pdbqt_editor.convert_to_flex_set()
                            flex_strs = []
                            if flex_set:
                                for chain_name_pos in flex_set:
                                    flex_strs.append(f'{chain_name_pos[0]}:{chain_name_pos[2]}')
                            update_dict['flexible'] = [gr.update(value=','.join(flex_strs))]
                        elif k in ['exhaustiveness', 'eval_poses']:
                            update_dict[k] = [gr.update(value=v)]
                    text = 'Protein loaded'
                except:
                    return ['Failed to load settings'] + [val for v in update_dict.values() for val in v]
            recursive_rm_file(working_dir)
            if self.pdbqt_editor is not None:
                update_dict['dock_status'] = [gr.update(interactive=True)]
        else:
            text = 'Protein removed'
        return [text] + [val for v in update_dict.values() for val in v]

    def process_ligand(self, files):
        if files:
            ligand_dir = os.path.join(os.path.join(working_dir, 'ligand_dir'))
            if not os.path.isdir(ligand_dir):
                os.mkdir(ligand_dir)
            for file in files:
                if file.endswith('.zip'):
                    with zipfile.ZipFile(file, 'r') as zip_f:
                        for f in zip_f.infolist():
                            if not f.is_dir() and f.filename.endswith('.pdbqt'):
                                extracted_path = os.path.join(ligand_dir, os.path.basename(f.filename))
                                zip_f.extract(f, ligand_dir)
                                os.rename(os.path.join(ligand_dir, f.filename), extracted_path)
                else:
                    base_name = os.path.basename(file.name)
                    os.rename(file, os.path.join(ligand_dir, base_name))
            for f_name in os.listdir(ligand_dir):
                n = os.path.join(ligand_dir, f_name)
                if os.path.isdir(n):
                    shutil.rmtree(n)
                elif f_name.startswith('.'):
                    os.remove(n)
            return f'{len(os.listdir(ligand_dir))} files uploaded.'

    def upload_docked_ligand(self, files):
        if files:
            docked_dir = os.path.join(working_dir, 'docked_dir')
            if not os.path.isdir(docked_dir):
                os.mkdir(docked_dir)
            cache_dir = os.path.join(docked_dir, 'cache_files')
            if not os.path.isdir(cache_dir):
                os.mkdir(cache_dir)
            for file in files:
                if file.endswith('.zip'):
                    with zipfile.ZipFile(file, 'r') as zip_f:
                        for f in zip_f.infolist():
                            file_name = f.filename
                            if not f.is_dir() and file_name.endswith('.pdbqt'):
                                if not f.filename.endswith(('_processed.pdbqt', '_rigid.pdbqt', '_flex.pdbqt', '.mds')):
                                    extracted_path = os.path.join(docked_dir, os.path.basename(file_name))
                                else:
                                    extracted_path = os.path.join(docked_dir, 'cache_files', os.path.basename(file_name))
                                zip_f.extract(f, docked_dir)
                                os.rename(os.path.join(docked_dir, file_name), extracted_path)
                else:
                    base_name = os.path.basename(file.name)
                    os.rename(file, os.path.join(docked_dir, base_name))
            for f_name in os.listdir(docked_dir):
                n = os.path.join(docked_dir, f_name)
                if f_name.startswith('.'):
                    os.remove(n)
            return f'{len(os.listdir(docked_dir))} files uploaded.'

    def _parse_flex_string(self, flex_res: str):
        flex_res_list = flex_res.split(',')
        for chain in self.pdbqt_editor.pdbqt_chain_dict:
            df = self.pdbqt_editor.pdbqt_chain_dict[chain]
            df['Flexible'] = [False] * len(df)
        for chain_res in flex_res_list:
            chain, res = chain_res.split(':')
            df = self.pdbqt_editor.pdbqt_chain_dict[chain]
            displayed_index = df[df['Display'] == True].index.to_list()
            if int(res) in displayed_index:
                df.loc[int(res), 'Flexible'] = True

    def _parse_display_string(self, display_res: str):
        display_res_list = display_res.split()
        if not display_res_list:
            return
        for display in display_res_list:
            chain, res = display.split(':')
            self.pdbqt_editor.update_display(chain, res)

    def zip_docked_files(self, check_status, zipped_name):
        if check_status:
            if not zipped_name:
                zipped_name = 'docked_result'
            zipped_file = os.path.join(working_dir, zipped_name + '.zip')
            docked_dir = os.path.join(working_dir, 'docked_dir')
            all_files = []
            for root, dirs, files in os.walk(docked_dir):
                for file in files:
                    file_path = os.path.join(root, file)
                    all_files.append(file_path)
            file_cnt = len(all_files)
            yield gr.update(), False, f'Progress ({0:{len(str(file_cnt))}}/{file_cnt})'
            num = 0
            with zipfile.ZipFile(zipped_file, 'w', zipfile.ZIP_LZMA) as zipf:
                for file in all_files:
                    zipf.write(file, os.path.relpath(file, docked_dir))
                    num += 1
                    yield gr.update(), gr.update(), f'Progress ({num:{len(str(file_cnt))}}/{file_cnt})'
            yield zipped_file, gr.update(), 'Zipping Done'
        else:
            yield gr.update(), gr.update(), gr.update()

    def check_docking_status(self):
        if not self.is_docking:
            return 'Not docking'
        else:
            with open('dock_energy.csv') as f:
                text = f.readlines()[1:]
            docked_files = len(text) - 1
            if docked_files != self.total_files:
                return f'Docking... ( {docked_files} / {self.total_files} )'
            else:
                return f'Docking Done!'

    def retrieve_energy(self, docked_pth: str):
        if not os.path.isfile(docked_pth):
            return ''
        with open(docked_pth) as f:
            pdbqt_str = f.read()
        for g in re.finditer(self.eng_regex, pdbqt_str):
            return g.group(1)

    def start_docking(self, vina_type,
                      center_x, center_y, center_z,
                      box_x   , box_y   , box_z   ,
                      exhaustiveness    , poses   , conc_num,
                      display_res       , flex_res,):
        vina_exec = os.path.join(working_dir, 'vina', 'gnina')
        docked_dir = os.path.join(working_dir, 'docked_dir')
        processed_protein_pth = os.path.join(docked_dir, 'cache_files', 'protein_processed.pdbqt')
        rigid_protein_pth = os.path.join(docked_dir, 'cache_files', 'protein_rigid.pdbqt')
        mds_pth = os.path.join(docked_dir, 'cache_files', 'protein_settings.mds')
        self._parse_display_string(display_res)
        flex_protein_pth = None
        flex_res = flex_res.replace(' ', '')
        if flex_res:
            flex_protein_pth = os.path.join(docked_dir, 'cache_files', 'protein_flex.pdbqt')
            self._parse_flex_string(flex_res)
        if flex_res is not None:
            self.pdbqt_editor.process_and_save_rigid_flex_processed_pdbqt(rigid_protein_pth, flex_protein_pth, processed_protein_pth)
        else:
            pdbqt_str = self.pdbqt_editor.convert_dict_to_pdbqt_text()
            with open(processed_protein_pth, 'w') as f:
                f.write(pdbqt_str)
            with open(rigid_protein_pth, 'w') as f:
                f.write(pdbqt_str)
        with open(mds_pth, 'wb') as f:
            param_dict = {'dock_center'   : {'x': center_x, 'y': center_y, 'z': center_z},
                          'dock_width'    : {'x': box_x   , 'y': box_y   , 'z': box_z   },
                          'exhaustiveness': exhaustiveness,
                          'eval_poses'    : poses,
                          'pdbqt_editor'  : self.pdbqt_editor.pdbqt_chain_dict}
            pickle.dump(param_dict, f)
        ligand_dir = os.path.join(working_dir, 'ligand_dir')
        self.total_files = sum(1 for f in os.listdir(ligand_dir) if f.endswith('.pdbqt') and not f.startswith('.'))
        docked_files = [f.rsplit('_docked.pdbqt')[0]+'.pdbqt' for f in os.listdir(docked_dir) if f.endswith('_docked.pdbqt') and not f.startswith('.')]
        non_docked_ligands = []
        docked_num = 0
        for f in os.listdir(ligand_dir):
            if f.endswith('.pdbqt') and not f.startswith('.'):
                if f not in docked_files:
                    non_docked_ligands.append(os.path.join(ligand_dir, f))
                else:
                    docked_num += 1
        if not non_docked_ligands:
            if not docked_num:
                return 'Ligands Not Uploaded!'
            else:
                return 'All Files Docked!'
        if any((box_x <= 0, box_y <= 0, box_z <= 0)):
            return 'Box size cannot be 0!'
        params = {'center': [center_x, center_y, center_z],
                  'box'   : [box_x   , box_y   , box_z   ],
                  'exhaustiveness': exhaustiveness        ,
                  'poses' : poses,
                  'vina_exec': vina_exec}
        with open('dock_energy.csv', 'a') as f:
            all_name_energies = [f'{docked_pth.rsplit("_docked.pdbqt")[0]},{self.retrieve_energy(os.path.join(docked_dir, docked_pth))}'
                                 for docked_pth in os.listdir(docked_dir) if docked_pth.endswith('_docked.pdbqt')
                                 and not docked_pth.startswith('.')]
            f.write('\n'.join(all_name_energies)+'\n')
        self.current_progress = docked_num
        overall_text = f'Starting Progress: {self.current_progress} / {self.total_files}\n'
        yield overall_text

        self.is_docking = True
        with ProcessPoolExecutor(max_workers=conc_num) as executor:
            futures = [executor.submit(single_vina_dock, rigid_protein_pth, flex_protein_pth,
                                       ligand_pth, docked_dir, params, vina_type) for
                       ligand_pth in non_docked_ligands]
            for f in as_completed(futures):
                result, name = f.result()

                if result:
                    rank_len = len(result)
                    eng_pose_strs = '\n'.join([f'Rank {i:<{rank_len}}: {eng} kcal/mol' for i, eng in enumerate(result, start=1)])
                    overall_text += f'{name}\n{eng_pose_strs}\n' + '\n'
                else:
                    overall_text += f'{name} Failed.\n'
                    overall_text += '\n'
                yield overall_text

        self.is_docking = False
        overall_text += 'Docking Done'
        return overall_text

    def new_dataframe(self):
        with open('dock_energy.csv') as f:
            text = f.readlines()[1:]
        array = [convert_row_to_string_float(l) for l in text if l.strip()]
        if array:
            text_len = len(str(self.total_files))
            return gr.DataFrame(value=array,
                                headers=['Name', 'Energy'],
                                type='array',
                                interactive=False,
                                label=f'{len(array):{text_len}}/{self.total_files}',)
        else:
            return gr.DataFrame(value=None,
                                headers=['Name', 'Energy'],
                                type='array',
                                interactive=False,
                                label='0/0',)

def fix_smina_pdbqt_file(out_pth: str, param_dict: dict,
                         smina_flex_res_re=r'AUTHOR.*+\n((.|\n)*?)CONECT.*\n'):
    if os.path.getsize(out_pth) == 0:
        os.remove(out_pth)
        if 'out_flex' in param_dict:
            os.remove(param_dict['out_flex'])
        return None
    else:
        all_flex_pos = []
        eng_list = []
        if 'out_flex' in param_dict:
            with open(param_dict['out_flex']) as f:
                flex_pdb = f.read()
            all_flex_pos = [g[0] for g in re.findall(smina_flex_res_re, flex_pdb)]
            os.remove(param_dict['out_flex'])
        flex_idx = 0

        with open(out_pth) as f:
            pdbqt_lines = []
            for line in f:
                if line.startswith('REMARK minimizedAffinity'):
                    affinity = line.split()[-1]
                    eng_list.append(affinity)
                    new_line = f'REMARK VINA RESULT: {affinity} 0.00 0.00\n'
                    pdbqt_lines.append(new_line)
                elif line.startswith('TORSDOF') and all_flex_pos:
                    pdbqt_lines.append('BEGIN_RES\n')
                    pdbqt_lines.append(all_flex_pos[flex_idx])
                    pdbqt_lines.append('END_RES\n')
                    flex_idx += 1
                else:
                    pdbqt_lines.append(line)

        with open(out_pth, 'w') as f:
            f.writelines(pdbqt_lines)

        return eng_list

def run_vina(receptor_pth, flex_pth, ligand_pth, docked_pth, param_dict, vina_type):
    cmd = [f'{param_dict["vina_exec"]}',
               '--receptor'      , f'{receptor_pth}'   ,
               '--ligand'        , f'{ligand_pth}'     ,
               '--center_x'      , f"{param_dict['center'][0]}",
               '--center_y'      , f"{param_dict['center'][1]}",
               '--center_z'      , f"{param_dict['center'][2]}",
               '--size_x'        , f"{param_dict['box'][0]}"   ,
               '--size_y'        , f"{param_dict['box'][1]}"   ,
               '--size_z'        , f"{param_dict['box'][2]}"   ,
               '--exhaustiveness', f"{param_dict['exhaustiveness']}",
               '--num_modes'     , f"{param_dict['poses']}",
               '--out'           , f"{docked_pth}"]
    if vina_type in ['smina', 'gnina']:
        cmd += ['--addH', 'False']
    if flex_pth is not None:
        cmd += ['--flex', f'{flex_pth}']
        if vina_type in ['smina', 'gnina']:
            cmd += ['--out_flex', f"{param_dict['out_flex']}"]
    if vina_type != 'gnina':
        cmd += ['--energy_range', '10']
    proc = subprocess.Popen(cmd, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL,
                            start_new_session=True)
    proc.wait()

def single_vina_dock(receptor_pth, flex_pth, ligand_pth, docked_dir, param_dict, vina_type):
    base_name = os.path.basename(ligand_pth).rsplit('.', 1)[0]
    docked_base_name = base_name + '_docked.pdbqt'
    docked_pth = os.path.join(docked_dir, docked_base_name)
    if vina_type in ['smina', 'gnina'] and flex_pth is not None:
        tmp_out_flex = os.path.join(docked_dir, base_name+'_flex.pdb')
        param_dict['out_flex'] = tmp_out_flex
    run_vina(receptor_pth, flex_pth, ligand_pth, docked_pth, param_dict, vina_type)
    if vina_type in ['smina', 'gnina']:
        eng_list = fix_smina_pdbqt_file(docked_pth, param_dict)
    else:
        eng_list = retrieve_energy(docked_pth)
    if eng_list:
        with open('dock_energy.csv', 'a') as f:
            f.write(f'{base_name},{eng_list[0]}\n')
    else:
        with open('dock_energy.csv', 'a') as f:
            f.write(f'{base_name},\n')
    return eng_list, base_name

def retrieve_energy(docked_pth: str):
    if not os.path.isfile(docked_pth):
        return []
    with open(docked_pth) as f:
        pdbqt_str = f.read()
    all_eng = re.findall(r'REMARK VINA RESULT:\s+(-?\d+\.\d+)', pdbqt_str)
    return all_eng


if __name__ == '__main__':
    interface = ProteinDockerInterface()
    interface.start_interface()